# SPECFEM Users Workshop -- Day 3 (Oct. 7, 2022)

## Part 3A: Seismic Imaging (Simple)

- In this notebook we will bring all of the previous material together run a seismic inversion to update a starting model   
- **Objective**: Manually update a velocity model and assess change in misfit   
- These instructions should be run from inside the Docker container, using Jupyter Lab (see *Docker Preamble* in Day 0)  

-----------

**Relevant Links:** 
- Today's Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_3a_imaging_simple.ipynb
- Completed Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_3a_imaging_simple.ipynb
- Day 0 Notebook (Container Testing): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_0_container_testing.ipynb
- Day 1A Notebook (Intro SPECFEM): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1a_intro_specfem2d.ipynb
- Day 1B Notebook (Fwd. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb
- Day 2A Notebook (Adj. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2a_kernels.ipynb

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.


## 1) Run a SeisFlows Example Problem

- We will use SeisFlows to automate an inversion workflow **up to** kernel generation, to save time  
- SeisFlows will run a forward simulation, calculate misfit, and run an adjoint simulation for one source and one receiver
- *Starting model*: homogeneous halfspace / *Target model*: perturbation checkerboard.
- **Objective**: Quickly generate an event kernel that we can use to manually update a 2D model  

In [ ]:
# Required Python packages for today's notebook
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from IPython.display import Image
from scipy.integrate import simps
from seisflows.tools.specfem import Model, read_fortran_binary, write_fortran_binary

In [ ]:
# Make sure we're in an empty working directory
! mkdir -p /home/scoped/work/day_3/sfexample_2
%cd /home/scoped/work/day_3/sfexample_2

# Run the example and stop after adjoint simulation
! seisflows examples setup 2 -r /home/scoped/specfem2d --event_id 1 --nsta 1 --niter 1 --with_mpi
! seisflows par stop_after evaluate_gradient_from_kernels
! seisflows submit

-----------

You will know that this workflow has completed successfully after the final log message   

```bash
2022-09-16 19:57:07 (I) | stop workflow at `stop_after`: evaluate_gradient_from_kernels
```

-----------
## 2) Manually Update the Starting Model

- SeisFlows has generated an event kernel for us  
- To provide context we will first visualize the starting and target models, as well as the kernel generated
- **Objective**: Use the gradient generated by SeisFlows to update the initial model. 
- **NOTE**: In SeisFlows, we treat models and gradients as linear arrays, so model update can be memory-intensive, but not CPU intensive.

>__MODEL UPDATE:__ The Model update equation *can be* as simple as: $m_{i+1} = m_{i} - \alpha \times g$   
Where $m_{i}$ is the current model, $m_{i+1}$ is the updated model, $g$ is the gradient and $\alpha$ is a scale factor.

In [ ]:
# Incase participants get lost, each numbered section 
# contains the absolute work dir path
%cd /home/scoped/work/day_3/sfexample_2


### a) Initial Model ($m_i$) and Target Model ($m_{true}$)

- The **initial/starting** model is a *homogeneous halfspace* model  
- It defines a region with P-wave velocity 5.8km/s and S-wave velocity 3.5km/s  
- The **target/true** model is a *perturbation checkerboard* model  
- Perturbations are roughly $\pm$10\% of the initial homogeneous halfspace Vp and Vs models  
- The target model is used to generate 'data', which mimics real-world observations  
- We have seen both of these models on Days 1 and 2  


In [ ]:
# Plot in the initial model Vs model
m_init = Model("output/MODEL_INIT")
print(f"INITIAL MODEL\n{m_init.model}")
m_init.plot2d("vs")

In [ ]:
# Plot in the target model Vs model
m_true = Model("output/MODEL_TRUE")
print(f"TARGET MODEL\n{m_true.model}")
m_true.plot2d("vs")

### b) Event Kernel

- The **event kernel** is generated through the interaction of the forward and adjoint wavefields  
- It is a volumetric quantity proportional to the gradient of the misfit function and provides a *search direction* for model update  
- Multiple event kernels summed together form the **misfit kernel**. Because we only have one event, the event and misfit kernels are the same  
- SeisFlows has already *smoothed* the kernel with a 2D Gaussian of vertical and horizontal half-widths equal to 5km

In [ ]:
# Load in Gradient binary files
gradient = Model("output/GRADIENT_01")

# Assign coordinate information which is used for plotting
# We do this because the Gradient output directory does not contain coordinate information, 
# so we need to grab it from the model
gradient.coordinates = {}
gradient.coordinates["x"] = m_init.coordinates["x"]
gradient.coordinates["z"] = m_init.coordinates["z"]

print(f"GRADIENT\n{gradient.model}")
gradient.plot2d("vs_kernel")

### c) Updated Model ($m_{i+1}$)

- We generate an updated model by scaling the gradient and adding the negative gradient to the initial model. 
- The gradient represents positive change, so we want to use the inverse gradient to reduce misfit.
- We use the equation $m_{i+1} = m_{i} - \alpha \times g$ to update the model.
- **Objective**: First we will **not** scale the gradient (i.e., $\alpha$==1)

In [ ]:
# We can use the SeisFlows Model class to update the model directly
m_update = m_init.copy()

# No scaling is applied here (alpha==1)
m_update.update(vector=m_init.vector - gradient.vector)

print(m_update.model)

# Plot the updated Vs model
m_update.plot2d("vs")

- We can see the gradient is **not** well scaled, so model updates only make slight changes to the Vs model (roughly $\pm.0001$m/s, see figure title). 
- In order to make significant changes to the starting model, we need the **scale** the gradient. 
- There are many algorithms which provide scaling estimates for the gradient. 

--------------

For an initial scaling guess, we can scale by: [G$^T$G]$^{-1}$ (i.e,. where G is the gradient)


In [ ]:
# Calculate GTG^-1
gtg = np.dot(gradient.vector, gradient.vector)
gtg_inverse = gtg ** -1 
print(f"GTG^-1 = {gtg_inverse:.2f}")

# Update the starting model with the scaled gradient
m_update = m_init.copy()
m_update.update(vector=m_init.vector - (gtg_inverse * gradient.vector))
print(m_update.model)

# Plot the model updated by the scaled gradient
m_update.plot2d("vs")

- The gradient is now more well scaled, and has updated our velocity model by at most 25m/s (see figure title). 
- Let's use this updated velocity model to generate **new** synthetics

--------------
## 3) Run Forward Simulations w/ Updated Model

- In Section 2c we generated an updated model, stored in the Python parameter `m_update`. 
- We want to use this updated model to generate a new set of synthetics
- Misfit with respect to the 'data' will be calculated using a waveform misfit (as in Day 2). 
- **Objective**: Determine if we have reduced waveform misfit through model update
- **NOTE:** We will work in `/home/scoped/work/day_3/specfem2d_workdir`. The following cells assume relative paths, so you must evaluate the '%cd' command below.

In [ ]:
# Setup block for our SPECFEM2D working directory
! mkdir -p /home/scoped/work/day_3/specfem2d_workdir
%cd /home/scoped/work/day_3/specfem2d_workdir

### a) Setup SPECFEM2D working directory
- We first set up a new SPECFEM2D working directory where we can run SPECFEM
- Our objective here is to run generate synthetics manually using our new model
- This task is very similar to what we did in Days 1 and 2

In [ ]:
# Symlink the binary files located in bin/
! ln -s /home/scoped/specfem2d/bin .
# Copy over the DATA/ directory with Par_file, SOURCE and STATIONS
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
# Pick a pre-defined Par_file set up for the homogeneous halfspace model
! cp DATA/Par_file_Tape2007_onerec DATA/Par_file
# Ensure we are using SOURCE #1
! cp DATA/SOURCE_001 DATA/SOURCE
# Ensure we are using only STATION #1
! head -1 DATA/STATIONS_checker > DATA/STATIONS
# Create the output directory required by SPECFEM
! mkdir OUTPUT_FILES

! ls

### b) Run SPECFEM2D to get initial model synthetics

- First we will run SPECFEM2D to get the initial model ($m_i$) synthetics
- Later we'll compare these to the updated model ($m_{i+1}$) synthetics


In [ ]:
# Tell SPECFEM to output FORTRAN binary files, and use the STATIONS file
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file use_existing_stations .true.

# Run SPECFEM with the homogeneous halfspace model, 1 source, 1 station
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver.txt

# A succesfully completed solver will write data files, expressed in the log
! tail -n 30 OUTPUT_FILES/output_solver.txt

### c) Set the Updated Model as the 'Current' Model

- SPECEFM2D needs to be able to find the updated model we created in Section 3c
- SPECFEM2D expects model files to be in the DATA/ directory. (**NOTE**: SPECFEM3D expects them to be in the `LOCAL_PATH` directory)
- We can use SeisFlows utilities to write these files in SPECFEM-readable formats
- We will OVERWRITE updated parameters (here Vp and Vs) and leave the remainder of the model the same
- **IMPORTANT**: The `Par_file` parameter 'MODEL' must be set to `gll`, which tells SPECFEM to read the updated model files we have written

In [ ]:
# First make sure that the original model files are stored somewhere safe
! mkdir MODEL_INIT
! cp -r DATA/*bin MODEL_INIT
! cp -r OUTPUT_FILES/*semd MODEL_INIT

print(f"parameters to be updated are: {m_update.parameters}")

# SeisFlows will overwrite the 'Vp' and 'Vs' binary files
m_update.write(path="DATA")

# Update the `Par_file` parameter 'MODEL' to tell SPECFEM to read model files from DATA/
! seisflows sempar -P DATA/Par_file model gll

### d) Run Forward Simulations with Updated Model

- We can now run `xmeshfem2D` and `xspecfem2D` to retrieve synthetics for our updated model
- Because we updated *model* files, we need to **rerun** `xmeshfem2D`. 
- Since we set `MODEL`==`gll`, SPECFEM2D will know to read the **updated** model files we just wrote (rather than reading internal files)
- The forward simulation **overwrites** files stored in `OUTPUT_FILES/`

In [ ]:
# Re-run the mesher and solver with the updated velocity model
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher_update.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver_update.txt

# Confirm that the updated model was used during the simulation
! head -236 OUTPUT_FILES/output_solver_update.txt | tail -n 15

# Make sure that the updated model files are stored somewhere safe
! mkdir MODEL_UPDATE
! cp -r DATA/*.bin MODEL_UPDATE
! cp -r OUTPUT_FILES/*semd MODEL_UPDATE

## 5) Comparing Synthetics for Initial and Updated Models

- We now have 2 synthetic seismograms. 1 for the initial model, 1 for the updated model
- In a real inversion, we would have 2$\times$N seismograms, corresonding to N stations 
- Our first goal is to quantify misfit of the new synthetics against the True model synthetics (checkerboard model)
- **Objective**: Determine if our updated model has reduced waveform misfit

In [ ]:
%cd /home/scoped/work/day_3/specfem2d_workdir

### a) Compare Initial and Update synthetics

- We first want to see if the model update changed the synthetic waveforms
- We'll re-use some code snippets from Day 2 to plot synthetics
- Plot colors are:
    - Initial model synthetics: *red*
    - Updated model synthetics: *purple*
    - Initial - Update (waveform difference): *blue*

In [ ]:
def plot_waveforms(t, data, labels, colors):
    """Re-usable function to plot synthetic waveforms"""
    for i, d in enumerate(data):
        plt.plot(t, d, c=colors[i], label=labels[i])
    plt.xlabel("Time [s]")
    plt.ylabel("Displacement [m]")
    plt.legend()  

In [ ]:
# Load in the initial, updated synthetics, calculate waveform difference
t_init, d_init = np.loadtxt("MODEL_INIT/AA.S000000.BXY.semd").T
t_update, d_update = np.loadtxt("MODEL_UPDATE/AA.S000000.BXY.semd").T
d_diff = d_init - d_update 

plot_waveforms(t=t_init, data=[d_init, d_update, d_diff],
               labels=["initial", "update", "difference"],
               colors=["r", "g", "b"])

plt.title("Initial vs. Updated Model Synthetics")
plt.show()

If we zoom into the time window where the two waveforms are different, we can better identify how the model update has affected the synthetic waveforms.
The time window of waveform difference (min/max amplitude of the blue wiggle) spans roughly *T0=60s* to *T1=85s*

In [ ]:
plot_waveforms(t=t_init, data=[d_init, d_update, d_diff],
               labels=["initial", "update", "difference"],
               colors=["r", "g", "b"])

plt.xlim([60, 85])
plt.title("Initial vs. Updated Model Synthetics")
plt.show()

In [ ]:
# Print min/max values to see if model update has affected waveform amplitude
print(f"INITIAL MODEL WAVEFORM: \n   MIN={d_init.min():.3f}; MAX={d_init.max():.3f}")
print(f"TARGET MODEL WAVEFORM:  \n   MIN={d_update.min():.3f}; MAX={d_update.max():.3f}")

#### Understanding waveform differences

- Visually the waveforms look almost the same. 
- Zooming in on the time window of waveform difference shows small waveform differences
- The updated synthetics (green) are slightly *delayed* with respect to the initial model synthetics (red)
- Updated synthetics (green) also have slightly larger minimum and maximum amplitudes w.r.t. initial model (red)


### b) Generate 'Data' using True/Target Model Synthetics

- We want to compare our two sets of synthetics against 'data'
- 'Data', generated by the checkerboard model, was already created by SeisFlows in Section 2
- However we perform this manually here to re-iterate using the input checkerboard model
- Misfit is defined as the waveform difference

In [ ]:
# This new par file OVERWRITES the changes we made previously 
! cp -f DATA/Par_file_Tape2007_132rec_checker DATA/Par_file

# Overwrite some default parameters
! seisflows sempar -P DATA/Par_file NSTEP 5000  # 4800 -> 5000 to match the other Par_file

# Ensure that SPECFEM can find the checkerboard model by name
! cp -f DATA/model_velocity.dat_checker DATA/proc000000_model_velocity.dat_input

# Re-run mesher and solver for the Target model
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher_true.txt
! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver_true.txt

# Copy waveforms and model into safe place
! mkdir -p MODEL_TRUE
! cp -r OUTPUT_FILES/*semd MODEL_TRUE

### c) Quantify Misfit between Initial/Update synthetics and Target model data

- We can now plot the target model data on top of the waveforms we showed earlier
- Target model synthetics (i.e., 'data') are generated using the checkerboard model
- Plot colors are:
    - Initial model synthetics: *red*
    - Updated model synthetics: *purple*
    - Initial - Update (waveform difference): *blue*
    - Target model: *black*

In [ ]:
# Load in Target model synthetics
t_true, d_true = np.loadtxt("MODEL_TRUE/AA.S000000.BXY.semd").T

# Plot the same as before
plot_waveforms(t=t_init, data=[d_init, d_update, d_diff, d_true],
               labels=["initial", "update", "difference", "target"],
               colors=["r", "g", "b", "k"])

plt.title("Initial, Updated, Target Model Synthetics")
plt.show()

In [ ]:
# Load in Target model synthetics
t_true, d_true = np.loadtxt("MODEL_TRUE/AA.S000000.BXY.semd").T

# Plot the same as before
plot_waveforms(t=t_init, data=[d_init, d_update, d_diff, d_true],
               labels=["initial", "update", "difference", "target"],
               colors=["r", "g", "b", "k"])

plt.title("Initial, Updated, Target Model Synthetics")
plt.xlim([60, 85])
plt.show()

#### Understanding waveform differences
- Target model synthetics (black) are *delayed* compared to initial model synthetics (red), by approximately 3s
- Target model synthetics (black) show *larger* amplitudes compared to initial model synthetics (red)
- By visual inspection, the updated model synthetics (green) appear to be moving *towards* the target synthetics (black)
- Visual inspection is a qualitative measure, we would prefer to **quantify** waveform misfit 

### c) Quantfiying Waveform Misfit

- Waveform misfit: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~$  (as in Day 2)
- In the equation above: $d(t)$ is 'data' and $s(t)$ is 'synthetics'
- **Objective**: Determine if the model update has reduced waveform misfit w.r.t initial model

In [ ]:
# Represents the time step, or `dt`. Same for all waveforms
dt = t_true[1] - t_true[0]  

# Use Simpson's rule for integration
misfit_init = 1/2 * simps((d_true - d_init)**2, dx=dt)
misfit_update = 1/2 * simps((d_true - d_update)**2, dx=dt)

print(f"misfit_init = {misfit_init:.3E}")
print(f"misfit_true = {misfit_update:.3E}")
print(f"init - true = {misfit_init - misfit_update:.3E}")
print(f"was misfit reduced? -> {bool(misfit_init > misfit_update)}")

---------------

**Mission accomplished!**
- We have successfully reduced the waveform misfit for a single source receiver pair by updating our velocity model.
- The misfit reduction is relatively small; this value is tied to the gradient and chosen scale factor $\alpha$
- We could increase the scale factor to achieve a more favorable misfit reduction, however if we go too far, we may end up increasing the misfit.
- Algorithmically, this step is defined as the `line search`, which attempts to search for an appropriate scale factor for model update
- There are many types of nonlinear optimization algorithms (e.g., gradient descent, L-BFGS) which are used to determine scale factors in an effort to minimize misfit
- **NOTE** each evaluation of a line search requires **re-running** the forward simulation with a trial model, it is therefore ideal to find the best performing algorithm to keep computational costs down 
- See e.g., [Modrak and Tromp (2016)](https://academic.oup.com/gji/article/206/3/1864/2583505) for exploration of how various algorithms perform in the context of seismic inverison
